In [1]:
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm

In [ ]:
with open('features_blacklist.txt', 'rt') as file_object:
    blacklist = [x.strip() for x in file_object.readlines()]
with open('features_nantozero.txt', 'rt') as file_object:
    nantozero = [x.strip() for x in file_object.readlines()]

def retrieve_table(conn, table):
    query = "select * from {}".format(table)
    table_df = (
        pd.read_sql(
            sql=query, 
            con=conn
        ).replace(
            to_replace='nan', value=np.nan
        ).astype(
            float, errors='ignore'
        )
    )
    return table_df

def retrieve_features(conn, table):
    image_query = "select TableNumber, ImageNumber, Metadata_Plate, Metadata_Well, Metadata_Site from image"
    features_df = pd.read_sql(
        sql=image_query, 
        con=conn
    ).astype(
        {
            'TableNumber': float,
            'ImageNumber': float
        }
    ).merge(
        retrieve_table(conn, table),
        on=[
            'TableNumber', 'ImageNumber'
        ],
        how="inner",
    ).drop(
        columns=blacklist,
        errors='ignore'
    ).drop(
        columns=['TableNumber', 'ImageNumber', 'ObjectNumber']
    )
    return features_df

sqlite_dir = "/data/bodipy/data/raw/sqlite"
stats_dir = "/data/bodipy/data/processed/summary_statistics"
quality_masks_dir = "/data/bodipy/data/metadata/quality_masks"
dst_path = "/data/bodipy/data/processed/features_replicates_median_plate_norm_nonPACs.csv"
cell_count_threshold = 50

layouts_df = pd.read_csv("/data/bodipy/data/metadata/layouts.csv")
plates_df = pd.read_csv("/data/bodipy/data/metadata/plates.csv")
plates = plates_df.loc[
    (plates_df.batch == 7) |\
    (plates_df.batch == 8) |\
    (plates_df.batch == 9),
    'plate'
].tolist()

for p, plate in enumerate(tqdm(plates)):
    #
    # Connect to sqlite database
    #
    sqlite_file = os.path.join(sqlite_dir, plate + '.sqlite')
    engine = create_engine("sqlite:///{}".format(sqlite_file))
    conn = engine.connect()

    #
    # Build plate mask based on cell count
    #
    query = "select Metadata_Plate, Metadata_Well, Metadata_Site, Count_Cells from image"
    mask_df = pd.read_sql(
        sql=query, 
        con=conn
    )
    mask_df = mask_df.loc[
        mask_df.Count_Cells > cell_count_threshold, :
    ].drop(
        columns='Count_Cells'
    )

    #
    # merge with site mask based on quality control
    #
    quality_mask_df = pd.read_csv(
        os.path.join(quality_masks_dir, plate + '.csv')
    )
    quality_mask_df = quality_mask_df.loc[
        quality_mask_df.isBad == False, :
    ].drop(
        columns=['isBad',]
    ).astype({'site': 'float64'})
    maks_df = mask_df.merge(
        quality_mask_df,
        how='inner',
        left_on=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'],
        right_on=['plate', 'well', 'site']
    ).drop(
        columns=['plate', 'well', 'site']
    )

    #
    # And mask based on patient ID (PAC, noPAC)
    #
    plate_layout_df = layouts_df.loc[layouts_df.plate == plate, :]
    plate_layout_df = plate_layout_df.loc[
        (plate_layout_df.patientID == 'SGBS') |\
        (plate_layout_df.patientID == 'hWAT') |\
        (plate_layout_df.patientID == 'hWAT-1') |\
        (plate_layout_df.patientID == 'hWAT-2') |\
        (plate_layout_df.patientID == 'hBAT') |\
        (plate_layout_df.patientID == 'hBAT-1') |\
        (plate_layout_df.patientID == 'hBAT-2'),
        ['plate', 'well']
    ]
    mask_df = mask_df.merge(
        plate_layout_df,
        how='inner',
        left_on=['Metadata_Plate', 'Metadata_Well'],
        right_on=['plate', 'well']
    ).drop(
        columns=['plate', 'well']
    )
    
    plate_df = pd.DataFrame()
    for i, table in enumerate(['Cells', 'Nuclei', 'Cytoplasm']):
        #
        # Extract cell profiler features from sqlite file
        #
        table_df = retrieve_features(conn, table)

        #
        # Transform NaN to zero and compute composite features
        #
        if table == 'Cells':
            for feature in nantozero:
                table_df.loc[:, feature].fillna(0, inplace=True)

            table_df['Cells_Children_BODIPYObjects_Count'] = table_df['Cells_Children_LargeBODIPYObjects_Count'] + table_df['Cells_Children_SmallBODIPYObjects_Count']

            table_df['Cells_LargeBODIPYObjects_AreaShape_Area'] = table_df['Cells_Mean_LargeBODIPYObjects_AreaShape_Area'] * table_df['Cells_Children_LargeBODIPYObjects_Count']
            table_df['Cells_SmallBODIPYObjects_AreaShape_Area'] = table_df['Cells_Mean_SmallBODIPYObjects_AreaShape_Area'] * table_df['Cells_Children_SmallBODIPYObjects_Count']
            table_df['Cells_BODIPYObjects_AreaShape_Area'] = table_df['Cells_LargeBODIPYObjects_AreaShape_Area'] + table_df['Cells_SmallBODIPYObjects_AreaShape_Area']

            table_df['Cells_LargeBODIPYObjects_Intensity_IntegratedIntensity_ER'] = table_df['Cells_Mean_LargeBODIPYObjects_Intensity_IntegratedIntensity_ER'] * table_df['Cells_Children_LargeBODIPYObjects_Count']
            table_df['Cells_SmallBODIPYObjects_Intensity_IntegratedIntensity_ER'] = table_df['Cells_Mean_SmallBODIPYObjects_Intensity_IntegratedIntensity_ER'] * table_df['Cells_Children_SmallBODIPYObjects_Count']
            table_df['Cells_BODIPYObjects_Intensity_IntegratedIntensity_ER'] = table_df['Cells_LargeBODIPYObjects_Intensity_IntegratedIntensity_ER'] + table_df['Cells_SmallBODIPYObjects_Intensity_IntegratedIntensity_ER']

        #
        # Mask features from excluded sites
        #
        table_df = table_df.merge(
            mask_df,
            on=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'],
            how='inner'
        )

        #
        # Adjust metadata plate value (may be not necessary)
        #
        table_df['Metadata_Plate'] = plate

        #
        # Normalize plate's table features
        #
        scaler = RobustScaler(quantile_range=(25.0, 75.0))
        features = [
            x 
            for x in table_df.columns.tolist() 
            if not x.startswith('Metadata')
        ]
        table_df.loc[:, features] = scaler.fit_transform(table_df.loc[:, features])

        #
        # Save plate normalization statistics
        #
        if i == 0:
            stats_df = pd.DataFrame(
                data=[scaler.center_, scaler.scale_],
                index=['Median', 'IQR'],
                columns=features
            )
        else:
            stats_df = pd.concat(
                [
                    pd.read_csv(
                        os.path.join(stats_dir, plate + 'noPAC_summary_statistics.csv'),
                        index_col=0
                    ),
                    pd.DataFrame(
                        data=[scaler.center_, scaler.scale_],
                        index=['Median', 'IQR'],
                        columns=features
                    )
                ],
                axis=1
            )
        stats_df.to_csv(
            os.path.join(stats_dir, plate + 'noPAC_summary_statistics.csv'),
            mode='w',
            index=True,
            header=True
        )

        #
        # Aggregate plate's table features
        #
        table_df = table_df.drop(
            columns=['Metadata_Site']
        ).merge(
            plates_df, 
            how='left', 
            left_on='Metadata_Plate', 
            right_on='plate'
        ).drop(
            columns=['notes', 'plate', 'batch']
        ).merge(
            layouts_df, 
            how='left', 
            left_on=['Metadata_Plate', 'Metadata_Well'],
            right_on=['plate', 'well']
        ).drop(
            columns=['well', 'plate', 'Metadata_Well']
        ).groupby(
            ["patientID", "day", "FFA", "cellType", "Metadata_Plate"]
        ).median().reset_index()

        #
        # Merge plate's table aggregated features
        #
        if i == 0:
            plate_df = table_df
        else:
            plate_df = plate_df.merge(
                table_df,
                how='inner',
                on=["patientID", "day", "FFA", "cellType", "Metadata_Plate"]
            )

    #
    # Save plate aggregated features
    #
    plate_df.to_csv(
        dst_path,
        mode='a' if p!=0 else 'w',
        index=False,
        header=(p==0)
    )

 45%|████▌     | 9/20 [2:20:54<3:16:59, 1074.52s/it]

In [3]:
query = "select Metadata_Plate, Metadata_Well, Metadata_Site, Count_Cells from image"
mask_df = pd.read_sql(
    sql=query, 
    con=conn
)
mask_df = mask_df.loc[
    mask_df.Count_Cells > cell_count_threshold, :
].drop(
    columns='Count_Cells'
)

In [5]:
mask_df.dtypes

Metadata_Plate     object
Metadata_Well      object
Metadata_Site     float64
dtype: object

In [10]:
quality_mask_df = pd.read_csv(
    os.path.join(quality_masks_dir, plate + '.csv')
)
quality_mask_df = quality_mask_df.loc[
    quality_mask_df.isBad == False, :
].drop(
    columns=['isBad',]
).astype({'site': 'float64'})

In [11]:
quality_mask_df.dtypes

well      object
site     float64
plate     object
dtype: object

In [8]:
maks_df = mask_df.merge(
    quality_mask_df,
    how='inner',
    left_on=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'],
    right_on=['plate', 'well', 'site']
).drop(
    columns=['plate', 'well', 'site']
)

In [9]:
mask_df.dtypes

Metadata_Plate     object
Metadata_Well      object
Metadata_Site     float64
dtype: object